<h1>Preparation</h1>

In [51]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [52]:
import pandas as pd
import numpy as np

In [53]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor

from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from joblib import dump

<h1>Dataframe read, data and shuffle</h1>

In [54]:
df = pd.read_csv("./ai1/datasets/dataset_concrete.csv")

In [55]:
df.dtypes

cement              float64
slag                float64
fly_ash             float64
water               float64
superplasticizer    float64
coarse_aggregate    float64
fine_aggregate      float64
age                   int64
strength            float64
dtype: object

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   cement            1030 non-null   float64
 1   slag              1030 non-null   float64
 2   fly_ash           1030 non-null   float64
 3   water             1030 non-null   float64
 4   superplasticizer  1030 non-null   float64
 5   coarse_aggregate  1030 non-null   float64
 6   fine_aggregate    1030 non-null   float64
 7   age               1030 non-null   int64  
 8   strength          1030 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 72.6 KB


In [57]:
df.describe(include = 'all')

,cement,slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [58]:
df = df.sample(frac = 1, random_state=2)
df.reset_index(drop=True, inplace = True)

<h1>Feature selection</h1>

In [59]:
features = ["cement", "slag", "fly_ash", "water","su", "age"]

#X=df[features].values
#y=df["strength"].values

In [60]:
dev_df, test_df = train_test_split(df, train_size=0.8, random_state=2)

In [61]:
# Extract the features but leave as a DataFrame
dev_X = dev_df[features]
test_X = test_df[features]

# Target values, converted to a 1D numpy array
dev_y = dev_df["strength"].values
test_y = test_df["strength"].values

In [62]:
ss = ShuffleSplit(n_splits=1, train_size=0.75, random_state=2)

<h1>Pipeline laying</h1>

In [76]:
preprocessor = ColumnTransformer([
    ("scaler",StandardScaler(),features)
], remainder = "drop")

<h3>Linear pipeline</h3>

In [77]:
linear_model = Pipeline([
    ("preprocessor", preprocessor),
    ("predictor", LinearRegression())])

<h3>Knn pipeline</h3>

In [78]:
knn_model = Pipeline([
    ("preprocessor", preprocessor),
    ("predictor", KNeighborsRegressor())])

In [79]:
param_grid = {"predictor__n_neighbors" : [1,2,3,4,5,6,7,8,9,10]}

<h2>We choose k fold, because we only have 1000 examples</h2>

In [80]:
# Create the grid search object which will find the best hyperparameter values based on validation error
gs = GridSearchCV(knn_model, param_grid, scoring="neg_mean_absolute_error", cv=ss)

# Run grid search by calling fit
gs.fit(dev_X, dev_y)

GridSearchCV(cv=ShuffleSplit(n_splits=1, random_state=2, test_size=None, train_size=0.75),
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('scaler',
                                                                         StandardScaler(),
                                                                         ['cement',
                                                                          'slag',
                                                                          'fly_ash',
                                                                          'water',
                                                                          'age'])])),
                                       ('predictor', KNeighborsRegressor())]),
             param_grid={'predictor__n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                    10]},
             scoring='neg_mean_absolute_error')

In [82]:
gs.best_params_, gs.best_score_

({'predictor__n_neighbors': 3}, -5.717022653721682)

<h1>Error estimation</h1>

In [84]:
knn_model.set_params(**gs.best_params_)
knn_model.fit(dev_X,dev_y)
mean_absolute_error(test_y, knn_model.predict(test_X))

5.429773462783172